# Sample Application

We are going to read a tetrahedral mesh from a .vtk file, send its surface into AxisVM to select points of applications for the applications of natural and essential boundary conditions, identify the selected items, build a finite element model, do a simple analysis and save the results in a simple, but powerful format.

The mesh for this tutorial is among the examples of AxisVM:

In [1]:
from axisvm import examples
import pyvista as pv
import numpy as np

from polymesh import PolyData, CartesianFrame, PointData
from polymesh.cells import TET4 as CellData
from polymesh.tri.triutils import edges_tri
from polymesh.topo import unique_topo_data, detach

vtkpath = examples.download_stand_vtk()
pd = PolyData.read(vtkpath)
c = pd.coords() / 1000
t = pd.topology()[:, :4]
c, t = detach(c, t)
csys = CartesianFrame(dim=3)

# pointdata and celldata
pd = PointData(coords=c, frame=csys)
cd = CellData(topo=t, frames=csys)

mesh = PolyData(pd, cd, frame=csys)

PolyData({})

Build the finite element model and perform a linear elastic analysis assuming small strains and displacements.

In [12]:
from sigmaepsilon.solid import Structure, PointData, FemMesh
from polymesh.space import StandardFrame
from sigmaepsilon.solid.fem.cells import TET4 as CellData
from neumann import repeat

# the global frame of the workspace
GlobalFrame = StandardFrame(dim=3)

# coordinates and topology
coords = mesh.coords()
topo = mesh.topology()

# essential boundary conditions
ebc_gids = [gid[id-1] for id in ebc_ids]
fixity = np.zeros((coords.shape[0], 6), dtype=bool)
fixity[ebc_gids, :3] = True
fixity[:, 3:] = True

# natural boundary conditions
F = 10
nbc_gids = [gid[id-1] for id in nbc_ids]
loads = np.zeros((coords.shape[0], 6))
loads[nbc_gids, 2] = -F

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = repeat(GlobalFrame.show(), topo.shape[0])
cd = CellData(topo=topo, frames=frames)

# define a stiffness matrix for Hooke's model
E = 12000.
nu = 0.2
Hooke = np.array([
    [1, nu, nu, 0, 0, 0], 
    [nu, 1, nu, 0, 0, 0],
    [nu, nu, 1, 0, 0, 0], 
    [0., 0, 0, (1-nu)/2, 0, 0],
    [0., 0, 0, 0, (1-nu)/2, 0],
    [0., 0, 0, 0, 0, (1-nu)/2]]) * (E / (1-nu**2))

mesh = FemMesh(pd, cd, model=Hooke, frame=GlobalFrame)

structure = Structure(mesh=mesh)

structure.linsolve()
dofsol = structure.nodal_dof_solution()

Plot the model, coloured with vertical displacements:

In [13]:
structure.mesh.plot(notebook=True, scalars=dofsol[:, 2], backend='k3d')

f:\GitHub\sigmaepsilon\.sigeps\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

Save the results:

In [14]:
import json
import numpy as np

np.save('coords', coords)
np.save('topo', topo)
np.save('hooke', Hooke)
np.save('nbc', loads)
np.save('ebc', fixity)
np.save('dofsol', dofsol)

data = {
    'materials' : {
        'steel' : 'hooke'
    },
    
    'mesh' : {
        'coordinates' : 'coords',
        'layout' : {
            'stand' : {
                'topology' : 'topo',
                'class' : 'TET4',
                'material' : 'steel'
                }
        }
    },
    
    'pointdata' : {
        'loads' : 'nbc',
        'fixity' : 'ebc',
        'dofsol' : 'dofsol'
    }
}

json.dumps(data, indent=4)

'{"materials": {"steel": "hooke"}, "mesh": {"coordinates": "coords", "layout": {"stand": {"topology": "topo", "class": "TET4", "material": "steel"}}}, "pointdata": {"loads": "nbc", "fixity": "ebc", "dofsol": "dofsol"}}'

In [15]:
with open("stand.json", "w") as outfile:
    json.dump(data, outfile)